In [1]:
import pandas as pd
import numpy as np

In [97]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv('emotion_chunk_1.csv')
data
num_rows = len(data)
print(num_rows)

50


In [98]:
data.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,is_relevant,preprocessed,sentiment,sentiment_scores,emotions,emotion_counts,emotions_nrc,sentiment _check,emotion_check
0,156787326,A Room of One's Own,NaN,A3U5AH52JL10DV,Eric Chuang,54/65,5,945129600,Provision for Feminist Future,"70 years have passed since the first publication of &quot;A Room of One's Own,&quot; and yet we have not seen as many dramatic changes, let alone improvements, in social mentality as early feminist thinkers like Virginia Woolf wished to provoke to establish women's new role as equal to men's. She argued that the reason why there was no female Shakespeare is not that women are biologically inferior to men but that there was simply no &quot;room&quot; for women to develop themselves, both metaphorically and realistically speaking. Therefore, in this book she encourages women to have a room of their own and a stable income to ensure a career. However, women, in the past as well as the present, have long been &quot;grounded&quot; by men. For a woman to have a room of her own and a stable income means that the woman is invading (from male chauvinistic viewpoint) men's territory, and this kind of behavior (and thoughts) is not to be allowed in male-dominated societies.Early feminist thinkers like Virginia Woolf provided later generations with iron-cast proof (as far as I'm concerned) that women are no &quot;second sex&quot; by pointing out the false discriminations men put against women for men's own convenience. (Ironically, I see men suffer as well from doing so.) Thinkers like Virginia Woolf provided &quot;rooms&quot; to develop feminist thoughts, and these rooms also provoke controversies and debates because feminist way of thinking is revolutionary. At any rate, there would be no improvements of women's role in society if there were no Virginia Woolf and other first-wave feminist thinkers.At the end of the twentieth century, in spite of the burgeoning &quot;industry&quot; of feminism, the real condition of women appears to be quite depressing. The real condition of women goes like this: &quot;During the last decades women's representation in education has grown enormously but so has our [their] participation in low paid and part-time work. So that, for example, the percentage of women in German higher education has doubled yet the degree of confidence German women express for women in non-traditional jobs is one of the lowest in Europe...,&quot; and &quot;similarly feminist literary criticism has created a lively and substantial body of work in the last decades but continues to exist in a hostile and often marginal academic place.&quot; (&quot;A Reader's Guide to Contemporary Feminist Literary Criticism&quot;, 1994: 290) One cannot fail to see (if one is willing to open his or her eyes) that we have made just a tiny bit of progress since the first-wave feminism. There is room for improvement, indeed, but people's ignorance of women's real position in society, women's subordinate educational, economical, and political conditions, and the overall social status of women being secondary, must first be recognized. In this sense, Virginia Woolf's &quot;A Room of One's Own&quot; appears to be especially inspiring.In my opinion, feminism is not only for or about women, it is also for and about men, because the world is composed of both sexes, and men suffer (without understanding, because of stupidity) from the traditional, male chauvinistic attitude as well. What is important is that feminist way of thinking is a breakthrough, revolutionary philosophy that challenges the way we perceive the world for centuries. &quot;A Room of One's Own&quot; opens my mind's eye; it is, no doubt, a classic that must be read.",True,70 year passed since first publication quot room one quot yet not seen many dramatic change let alone improvement social mentality early feminist thinker like virginia woolf wished provoke es

# getting text ready to be classified  

In [ ]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from scipy.special import softmax
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from joblib import Parallel, delayed
import re
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
data['preprocessed'] = data['review/text'].str.lower()
def remove_punctuation(text):
    if pd.isna(text):  # Check if the value is NaN
        return ""
    else:
        return re.sub(r'[^\w\s]', ' ', text)
data['preprocessed'] = data['preprocessed'].apply(remove_punctuation)
data['preprocessed'] = data['preprocessed'].apply(lambda x: re.sub(r'(\w)\1{2,}', r'\1\1', x))
data['preprocessed'] = data['preprocessed'].apply(lambda x: re.sub(r'\*{2}.*?\*{2}|\(.*?\)', '', x))
data['preprocessed'] = data['preprocessed'].apply(lambda x: re.sub(r'"(.*?)"', '', x))
data['preprocessed'] = data['preprocessed'].apply(lambda x: re.sub(r'\b(?:https?://|www\.)\S+\b|[\w\.-]+@[\w\.-]+\.\w+', '', x))
data['preprocessed'] = data['preprocessed'].apply(lambda x: re.sub(r'\s+', ' ', x))
stop_words = set(stopwords.words('english'))
negative_stop_words = set(['not'])
final_stop_words = stop_words - negative_stop_words
def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word not in final_stop_words])
lemmatizer = WordNetLemmatizer()
def lemma_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

def preprocess_text(text):
    text = remove_stop_words(text)
    text = lemma_text(text)
    return text

data['preprocessed'] = Parallel(n_jobs=-1)(delayed(preprocess_text)(text) for text in data['preprocessed'])

In [7]:
# Load the pre-trained model and tokenizer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
def get_sentiment(text):
    if not text:
        return 'roberta_neutral', 0.0
    max_length = 512
    texts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    scores_sum = {
        'roberta_negative' : 0,
        'roberta_neutral' : 0,
        'roberta_positive' : 0
    }
    for txt in texts:
        encoded_text = tokenizer(txt, return_tensors='pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'roberta_negative' : scores[0],
            'roberta_neutral' : scores[1],
            'roberta_positive' : scores[2]
        }
        for key in scores_sum.keys():
            scores_sum[key] += scores_dict[key]
    
    # Compute the average scores
    scores_avg = {key: val / len(texts) for key, val in scores_sum.items()}
    # Get the sentiment with the maximum average score
    overall_sentiment = max(scores_avg, key=scores_avg.get)
    
    return overall_sentiment, scores_avg[overall_sentiment]
data['sentiment'], data['sentiment_scores'] = zip(*data['preprocessed'].apply(get_sentiment))



C:\Users\HP\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


NameError: name 'rates' is not defined

In [8]:
data.head()


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,aboutTheBookOrNot,whileAfter,preprocessed,sentiment,sentiment_scores
0,B000OVMRLA,The Last Unicorn,NaN,A144JRBX4T1N98,Bill Rambo,0/1,5.0,1252713600,"Unicorns don't change, but cultures do.","I was captivated by Beagle's Last Unicorn in university, and just decided that I needed a new copy. I hadn't read it in years, and found it just as enthralling as it was over thirty years ago.But I was surprised at how difficult it was to ignore the presence of the Red Bull...I mean, the energy drink, which kept diluting the serious and mysterious fear due to Haggard's bull. I've come to terms with my last name, and the connotations recent acquaintances attach to me because of it, but I was sad that such a great piece of literature had been sullied by a cultural phenomenon.",most of it about the book but the reader loved nd enjoyed it,while,captivated beagle last unicorn university decided needed new copy read year found enthralling thirty year ago surprised difficult ignore presence red bull mean energy drink kept diluting serious mysterious fear due haggard bull come term last name connotation recent acquaintance attach sad great piece literature sullied cultural phenomenon,roberta_negative,0.574266
1,1578152445,Great Expectations,NaN,A1ALP9L1SW6NJ0,Benjamin,0/3,2.0,1014163200,A short story told in 400 pages.,"This book could have made a wonderful short story, but as a full length novel it lacks both and entertaining plot and characters that one can truly empathize with.","about the book,book is not entertainning ,so she didn't enjoy it",while,book could made wonderful short story full length novel lack entertaining plot character one truly empathize,roberta_positive,0.845917
2,B000CE8QSM,Tempted (Book Club Edition),NaN,A3LEH416AV3VTU,Regan,0/0,5.0,1326499200,Captivating Story of Scottish Love in the Borderlands,"In a complex, lusty tale of feuding Scottish clans during the time of James IV of Scotland and Henry VIII of England, Henley serves up a story that will grab you from the first page and will not let you go. I loved it. Henley has the ability to capture not only the time period, but also the culture of the borderlands between Scotland and England and the difficult but wonderful relationship between two stubborn, courageous people. I highly recommend this one.Set in Scotland in 1512, TEMPTED is the story of clans Douglas, Hamilton and Kennedy. Beautiful Valentina Kennedy, the oldest of the Kennedy daughters, was known as Flaming Tina for her glorious red-gold hair and her feisty temper. She was raised with her sister and brothers in Castle Doon in Ayr on the Southwest coast of Scotland where her father ruled as the Lord of Galloway. Tina despaired the lot of women that prevented them from enjoying male pursuits, and she frequently joined her brothers in their activities. But when she turns 17, her father insists she say yes to one of her many suitors. Tina thinks to marry the Hamilton heir but is happy when the Gypsy tells her there will be no wedding for her this year. In a surprising turn of events, she is hand fasted to her enemy, the powerful Black Ram Douglas, who she considers to be a brute of a man. Tina intends to have her revenge...Henley develops some wonderful characters, including the ghosts of two star-crossed lovers from the past (another Kennedy and Douglas pairing), who constantly entertain. She paints vivid word pictures of the food, clothing and castles that really drawn you in and help you experience the times--and she does it very well. In a fitting climax to the story, she superbly describes the battle between the English and Scots at Flodden where Scotland went down in terrible defeat.This is going on my Best Scottish/Highlander Historical Romances list! The second in the 2-book Clan Kennedy Saga is THE BORDER HOSTAGE.","about the book, she enjoyed reading it and loved it",while,complex lusty tale feudin

# using the emotion.txt mintioned in the paper

In [9]:
import pandas as pd
import nltk
from collections import Counter

def detect_emotions(tokens):
    emotion_list = []
    with open('emotions.txt', 'r') as file:
        for line in file:
            clear_line = line.replace("\n", '').replace(",", '').replace("'", '').strip()
            word, emotion = clear_line.split(':')
            if word in tokens:
                emotion_list.append(emotion)
    return emotion_list

# Apply the emotion detection function to each row
data['emotions'] = data['review/text'].apply(detect_emotions)

# Count the emotions in each row
data['emotion_counts'] = data['emotions'].apply(Counter)

print(data.head(30))


            Id  \
0   B000OVMRLA   
1   1578152445   
2   B000CE8QSM   
3   B0006DXW74   
4   B000GYE7EA   
5   0830736697   
6   B0006DGNQ6   
7   078381514X   
8   0375502505   
9   0961390158   
10  B000NR8VX8   
11  B000PCESRE   
12  B000H1XWQQ   
13  B000CEXDTO   
14  1593350767   
15  0747553688   
16  B000GSDG8E   
17  B000KO327E   
18  B000RB0WGO   
19  069452607X   
20  B0000X8R9I   
21  0613647874   
22  0613176375   
23  0786222859   
24  0613103572   
25  B000E39CZW   
26  B000JFA9N4   
27  1587240149   
28  0465033873   
29  1400063817   

                                                                                                                                                     Title  \
0                                                                                                                                         The Last Unicorn   
1                                                                                                                            

# Apply NRC different emotion lixicon with 6 emotions

In [10]:
from collections import defaultdict

emotion_lexicon = pd.read_csv('NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt', names=["word", "emotion", "association"], sep='\t')

def get_emotions_nrc(text):
    emotions = defaultdict(int)
    for word in word_tokenize(text):
        word_emotions = emotion_lexicon[emotion_lexicon['word'] == word]
        for _, row in word_emotions.iterrows():
            if row['association'] == 1:
                emotions[row['emotion']] += 1
    return dict(emotions)

data['emotions_nrc'] = data['review/text'].apply(get_emotions_nrc)

In [11]:
data.head(20)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,aboutTheBookOrNot,whileAfter,preprocessed,sentiment,sentiment_scores,emotions,emotion_counts,emotions_nrc
0,B000OVMRLA,The Last Unicorn,NaN,A144JRBX4T1N98,Bill Rambo,0/1,5.0,1252713600,"Unicorns don't change, but cultures do.","I was captivated by Beagle's Last Unicorn in university, and just decided that I needed a new copy. I hadn't read it in years, and found it just as enthralling as it was over thirty years ago.But I was surprised at how difficult it was to ignore the presence of the Red Bull...I mean, the energy drink, which kept diluting the serious and mysterious fear due to Haggard's bull. I've come to terms with my last name, and the connotations recent acquaintances attach to me because of it, but I was sad that such a great piece of literature had been sullied by a cultural phenomenon.",most of it about the book but the reader loved nd enjoyed it,while,captivated beagle last unicorn university decided needed new copy read year found enthralling thirty year ago surprised difficult ignore presence red bull mean energy drink kept diluting serious mysterious fear due haggard bull come term last name connotation recent acquaintance attach sad great piece literature sullied cultural phenomenon,roberta_negative,0.574266,"[ attached, attracted, sad, surprise, happy]","{' attached': 1, ' attracted': 1, ' sad': 1, ' surprise': 1, ' happy': 1}","{'anticipation': 2, 'positive': 3, 'negative': 3, 'joy': 1, 'trust': 1, 'surprise': 2, 'fear': 3, 'anger': 1}"
1,1578152445,Great Expectations,NaN,A1ALP9L1SW6NJ0,Benjamin,0/3,2.0,1014163200,A short story told in 400 pages.,"This book could have made a wonderful short story, but as a full length novel it lacks both and entertaining plot and characters that one can truly empathize with.","about the book,book is not entertainning ,so she didn't enjoy it",while,book could made wonderful short story full length novel lack entertaining plot character one truly empathize,roberta_positive,0.845917,[ angry],{' angry': 1},"{'joy': 2, 'positive': 3, 'surprise': 1, 'trust': 1, 'anticipation': 1}"
2,B000CE8QSM,Tempted (Book Club Edition),NaN,A3LEH416AV3VTU,Regan,0/0,5.0,1326499200,Captivating Story of Scottish Love in the Borderlands,"In a complex, lusty tale of feuding Scottish clans during the time of James IV of Scotland and Henry VIII of England, Henley serves up a story that will grab you from the first page and will not let you go. I loved it. Henley has the ability to capture not only the time period, but also the culture of the borderlands between Scotland and England and the difficult but wonderful relationship between two stubborn, courageous people. I highly recommend this one.Set in Scotland in 1512, TEMPTED is the story of clans Douglas, Hamilton and Kennedy. Beautiful Valentina Kennedy, the oldest of the Kennedy daughters, was known as Flaming Tina for her glorious red-gold hair and her feisty temper. She was raised with her sister and brothers in Castle Doon in Ayr on the Southwest coast of Scotland where her father ruled as the Lord of Galloway. Tina despaired the lot of women that prevented them from enjoying male pursuits, and she frequently joined her brothers in their activities. But when she turns 17, her father insists she say yes to one of her many suitors. Tina thinks to marry the Hamilton heir but is happy when the Gypsy tells her there will be no wedding for her this year. In a surprising turn of events, she is hand fasted to her enemy, the powerful Black Ram Douglas, who she considers to be a brute of a man. Tina intends to have her revenge...Henley develops some wonderful characters, including the ghosts of two star-crossed lovers from the past (another Kennedy and Douglas pairing), who constantly entertain. She paints vivid word pictures of the food, clothing and castles that really drawn you in and help you experience the times--and she does it very well. In a fi

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
data = pd.read_csv('emotionschunk_5.csv')
data
num_rows = len(data)
print(num_rows)

50


In [3]:
data


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,aboutTheBookOrNot,whileAfter,preprocessed,sentiment,sentiment_scores,emotions,emotion_counts,emotions_nrc
0,B000OVMRLA,The Last Unicorn,NaN,A144JRBX4T1N98,Bill Rambo,0/1,5.0,1252713600,"Unicorns don't change, but cultures do.","I was captivated by Beagle's Last Unicorn in university, and just decided that I needed a new copy. I hadn't read it in years, and found it just as enthralling as it was over thirty years ago.But I was surprised at how difficult it was to ignore the presence of the Red Bull...I mean, the energy drink, which kept diluting the serious and mysterious fear due to Haggard's bull. I've come to terms with my last name, and the connotations recent acquaintances attach to me because of it, but I was sad that such a great piece of literature had been sullied by a cultural phenomenon.",most of it about the book but the reader loved nd enjoyed it,while,captivated beagle last unicorn university decided needed new copy read year found enthralling thirty year ago surprised difficult ignore presence red bull mean energy drink kept diluting serious mysterious fear due haggard bull come term last name connotation recent acquaintance attach sad great piece literature sullied cultural phenomenon,roberta_negative,0.574266,"[' attached', ' attracted', ' sad', ' surprise', ' happy']","Counter({' attached': 1, ' attracted': 1, ' sad': 1, ' surprise': 1, ' happy': 1})","{'anticipation': 2, 'positive': 3, 'negative': 3, 'joy': 1, 'trust': 1, 'surprise': 2, 'fear': 3, 'anger': 1}"
1,1578152445,Great Expectations,NaN,A1ALP9L1SW6NJ0,Benjamin,0/3,2.0,1014163200,A short story told in 400 pages.,"This book could have made a wonderful short story, but as a full length novel it lacks both and entertaining plot and characters that one can truly empathize with.","about the book,book is not entertainning ,so she didn't enjoy it",while,book could made wonderful short story full length novel lack entertaining plot character one truly empathize,roberta_positive,0.845917,[' angry'],Counter({' angry': 1}),"{'joy': 2, 'positive': 3, 'surprise': 1, 'trust': 1, 'anticipation': 1}"
2,B000CE8QSM,Tempted (Book Club Edition),NaN,A3LEH416AV3VTU,Regan,0/0,5.0,1326499200,Captivating Story of Scottish Love in the Borderlands,"In a complex, lusty tale of feuding Scottish clans during the time of James IV of Scotland and Henry VIII of England, Henley serves up a story that will grab you from the first page and will not let you go. I loved it. Henley has the ability to capture not only the time period, but also the culture of the borderlands between Scotland and England and the difficult but wonderful relationship between two stubborn, courageous people. I highly recommend this one.Set in Scotland in 1512, TEMPTED is the story of clans Douglas, Hamilton and Kennedy. Beautiful Valentina Kennedy, the oldest of the Kennedy daughters, was known as Flaming Tina for her glorious red-gold hair and her feisty temper. She was raised with her sister and brothers in Castle Doon in Ayr on the Southwest coast of Scotland where her father ruled as the Lord of Galloway. Tina despaired the lot of women that prevented them from enjoying male pursuits, and she frequently joined her brothers in their activities. But when she turns 17, her father insists she say yes to one of her many suitors. Tina thinks to marry the Hamilton heir but is happy when the Gypsy tells her there will be no wedding for her this year. In a surprising turn of events, she is hand fasted to her enemy, the powerful Black Ram Douglas, who she considers to be a brute of a man. Tina intends to have her revenge...Henley develops some wonderful characters, including the ghosts of two star-crossed lovers from the past (another Kennedy and Douglas pairing), who constantly entertain. She paints vivid word pictures of the food, clothing and castles that really drawn you in and help you experience the times--

In [10]:
data['check roberta'] = Roberta
data['check_emofile'] = Emotiontxtfile
data['check_nrc'] = NRC

NameError: name 'Roberta' is not defined

# we took chunks of the output and rate it manually 
* first chunk

In [12]:
relevance_counts = data['relevence'].value_counts()

# Get counts for True and False
true_count = relevance_counts.get(True, 0)
false_count = relevance_counts.get(False, 0)

print(f"Number of True: {true_count}")
print(f"Number of False: {false_count}")

Number of True: 43
Number of False: 7


In [19]:
filtered_data = data[data['relevence'] == True]

# Further filter out None and empty string values from check_emofile
relevance_counts = filtered_data['check roberta'].value_counts()

# Get counts for True and False
true_count = relevance_counts.get(True, 0)
false_count = relevance_counts.get(False, 0)

print(f"True check roberta: {true_count}")
print(f"alse check roberta: {false_count}")

# Count the number of True and False values
relevance_counts = filtered_data['check_emofile'].value_counts()

# Get counts for True and False
true_count = relevance_counts.get(True, 0)
false_count = relevance_counts.get(False, 0)

print(f"false of emotion txt: {true_count}")
print(f"false emotion txt: {false_count}")

relevance_counts = filtered_data['check_nrc'].value_counts()
# Get counts for True and False
true_count = relevance_counts.get(True, 0)
false_count = relevance_counts.get(False, 0)

print(f"True nrc: {true_count}")
print(f"False nrc: {false_count}")

True check roberta: 37
alse check roberta: 6
false of emotion txt: 25
false emotion txt: 18
True nrc: 34
False nrc: 9


* second chunk

In [13]:
relevance_counts = data['check roberta'].value_counts()

# Get counts for True and False
true_count = relevance_counts.get(True, 0)
false_count = relevance_counts.get(False, 0)

print(f"Number of True: {true_count}")
print(f"Number of False: {false_count}")

Number of True: 43
Number of False: 7


In [14]:
relevance_counts = data['check_emofile'].value_counts()
# Get counts for True and False
true_count = relevance_counts.get(True, 0)
false_count = relevance_counts.get(False, 0)

print(f"Number of True: {true_count}")
print(f"Number of False: {false_count}")

Number of True: 28
Number of False: 22


In [15]:

relevance_counts = data['check_nrc'].value_counts()
# Get counts for True and False
true_count = relevance_counts.get(True, 0)
false_count = relevance_counts.get(False, 0)

print(f"Number of True: {true_count}")
print(f"Number of False: {false_count}")

Number of True: 38
Number of False: 12


In [6]:
data

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,relevence,preprocessed,sentiment,sentiment_scores,emotions,emotion_counts,emotions_nrc,check roberta,check_emofile,check_nrc
0,0440242177,"To Tame a Highland Warrior (The Highlander Series, Book 2)",NaN,A70W2QXT6LGEX,J.M. Meyer,12-Nov,5,948844800,Don't miss out on reading this one!,"I absolutely adored Karen's new book, &quot;To Tame a Highland Warrior&quot;. It was a book that I truly could not put down. Karen writes in such a way that &quot;bam&quot; she grabs your attention in the very beginning and pulls you right into the story itself. While reading this novel, I felt a real familiarity with the two main characters, Grimm and Jillian. She really gets into setting up the background (but doesn't pound it - background info, that is - to death like so many other authors do, so much that you want to put the book down right then) for her characters so that you feel like you know them personally. You can really &quot;feel&quot; the characters' emotions and the conflicts going on between them. I have to say that it's so very easy with Karen Moning's writing style to get totally caught up and lost in the story line. When I read her first book &quot;Beyond the Highland Mist&quot;, I didn't think that a romance novel could get any better. Boy was I wrong. This novel, in my opinion, far surpasses the last, which like I said above, didn't think could get any better! I really loved everything about her latest novel - especially the way she grabs your attention from the start and keeps it the whole way through! Can't wait to read her next one!",True,absolutely adored karen new book quot tame highland warrior quot book truly could not put karen writes way quot bam quot grab attention beginning pull right story reading novel felt real familiarity two main character grimm jillian really get setting background pound background info death like many author much want put book right character feel like know personally really quot feel quot character emotion conflict going say easy karen moning writing style get totally caught lost story line read first book quot beyond highland mist quot think romance novel could get better boy wrong novel opinion far surpasses last like said think could get better really loved everything latest novel especially way grab attention start keep whole way wait read next one,roberta_positive,0.811410,"[' loved', ' sad', ' lost', ' attached', ' loved']","Counter({' loved': 2, ' sad': 1, ' lost': 1, ' attached': 1})","{'positive': 7, 'trust': 3, 'anticipation': 5, 'joy': 2, 'anger': 2, 'negative': 5, 'disgust': 1, 'fear': 2, 'sadness': 3, 'surprise': 2}",True,True,True
1,0156078805,Architecture Of The Arkansas Ozarks,NaN,A5XPVYRF6LNDS,"J. Cagle ""xClimber""",8-Aug,5,1016236800,Classic Work of Regional Fiction,"When I first read this book in the early seventies, I was in my late teens. I have since re-read it a number of times, and each time it reveals more of the nuance of character inherent to the individual hill-williams Mr. Harrington profiles in this classic novel. Having lived in Arkansas since the age of 11 I felt as if I'd grown up with many of the characters he portrays. If you enjoy ludicrous situations and quirky characters read this book NOW! It should be listed on the Norman Cousins health plan...",True,first read book early seventy late teen since read number time time reveals nuance character inherent individual hill williams mr harrington profile classic novel lived arkansas since age 11 felt grown many character portrays enjoy ludicrous situation quirky character read book listed norman cousin health plan,roberta_neutral,0.578651,[' apathetic'],Counter({' apathetic': 1}),"{'negative': 3, 'sadness': 1, 'positive': 3, 'anticipation': 3, 'joy': 1, 'trust': 1}",False,False,True
2,0192817728,The Secret Garden (Worlds Classics),NaN,A1SK6CN4H2TBZ,Kelly,0/0,5,1358294400,Feel good story,Fabulous book! Fast

# To enhance the emotion file we took each adjective and found the sensets of it and apply it on the code 

In [46]:
import nltk
from nltk.corpus import wordnet
import ast
nltk.download('wordnet')
nltk.download('omw-1.4')
emotion_lexicon = {}
with open('emotions.txt') as f:
    for line in f:
        word, emotion = line.strip().split(': ')
        emotion_lexicon[word.strip("'")] = emotion.strip("',")
def find_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return synonyms
expanded_lexicon = emotion_lexicon.copy()
for word, emotion in emotion_lexicon.items():
    synonyms = find_synonyms(word)
    for synonym in synonyms:
        expanded_lexicon[synonym] = emotion
with open('expanded_emotions.txt', 'w') as f:
    for word, emotion in expanded_lexicon.items():
        f.write(f"'{word}': '{emotion}',\n")




[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
data = pd.read_csv('emotion_chunk_1.csv')

# Handles negation before 

In [50]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd

stop_words = set(stopwords.words('english'))
negative_stop_words = set(['not'])
final_stop_words = stop_words - negative_stop_words
def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word not in final_stop_words])
data['cleaned_review'] = data['review/text'].apply(remove_stop_words)
def preprocess_text_with_negation(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    text = remove_stop_words(text)
    tokens = word_tokenize(text)
    # Handle negations
    negation_words = set(['not', 'no', 'never', 'none'])
    negated = False
    processed_tokens = []
    
    for token in tokens:
        if token in negation_words:
            negated = True
            continue
        
        if negated:
            processed_tokens.append(token + '_NEG')
            negated = False
        else:
            processed_tokens.append(token)
    
    return ' '.join(processed_tokens)

data['cleaned_review'] = data['review/text'].apply(preprocess_text_with_negation)


NameError: name 'stop_words' is not defined

In [51]:
# Load the expanded emotion lexicon
expanded_lexicon = {}
with open('expanded_emotions.txt') as f:
    for line in f:
        word, emotion = line.strip().split(': ')
        expanded_lexicon[word.strip("'")] = emotion.strip("',")

# Function to classify text based on the expanded lexicon
def classify_emotions_with_negation(text):
    emotions = []
    for word in text.split():
        if word in expanded_lexicon:
            emotions.append(expanded_lexicon[word])
        elif word.endswith('_NEG') and word[:-4] in expanded_lexicon:
            # Handle negated words
            original_emotion = expanded_lexicon[word[:-4]]
            negated_emotion = f"not_{original_emotion}"
            emotions.append(negated_emotion)
    return emotions



In [53]:
sample_sentences = [
    "I am happy",
    "I am not happy",
    "I enjoyed this book",
    "is none happy"
]

for sentence in sample_sentences:
    processed = preprocess_text_with_negation(sentence)
    emotions = classify_emotions_with_negation(processed)
    print(f"Sentence: {sentence}")
    print(f"Processed: {processed}")
    print(f"Emotions: {emotions}\n")

Sentence: I am happy
Processed: i am happy
Emotions: ['happy']

Sentence: I am not happy
Processed: i am happy_NEG
Emotions: ['not_happy']

Sentence: I enjoyed this book
Processed: i enjoyed this book
Emotions: []

Sentence: is none happy
Processed: is happy_NEG
Emotions: ['not_happy']



In [38]:
data.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,...,preprocessed,sentiment,sentiment_scores,emotions,emotion_counts,emotions_nrc,check roberta,check_emofile,check_nrc,cleaned_review
0,0440242177,"To Tame a Highland Warrior (The Highlander Series, Book 2)",NaN,A70W2QXT6LGEX,J.M. Meyer,12-Nov,5,948844800,Don't miss out on reading this one!,"I absolutely adored Karen's new book, &quot;To Tame a Highland Warrior&quot;. It was a book that I truly could not put down. Karen writes in such a way that &quot;bam&quot; she grabs your attention in the very beginning and pulls you right into the story itself. While reading this novel, I felt a real familiarity with the two main characters, Grimm and Jillian. She really gets into setting up the background (but doesn't pound it - background info, that is - to death like so many other authors do, so much that you want to put the book down right then) for her characters so that you feel like you know them personally. You can really &quot;feel&quot; the characters' emotions and the conflicts going on between them. I have to say that it's so very easy with Karen Moning's writing style to get totally caught up and lost in the story line. When I read her first book &quot;Beyond the Highland Mist&quot;, I didn't think that a romance novel could get any better. Boy was I wrong. This novel, in my opinion, far surpasses the last, which like I said above, didn't think could get any better! I really loved everything about her latest novel - especially the way she grabs your attention from the start and keeps it the whole way through! Can't wait to read her next one!",...,absolutely adored karen new book quot tame highland warrior quot book truly could not put karen writes way quot bam quot grab attention beginning pull right story reading novel felt real familiarity two main character grimm jillian really get setting background pound background info death like many author much want put book right character feel like know personally really quot feel quot character emotion conflict going say easy karen moning writing style get totally caught lost story line read first book quot beyond highland mist quot think romance novel could get better boy wrong novel opinion far surpasses last like said think could get better really loved everything latest novel especially way grab attention start keep whole way wait read next one,roberta_positive,0.811410,"[' loved', ' sad', ' lost', ' attached', ' loved']","Counter({' loved': 2, ' sad': 1, ' lost': 1, ' attached': 1})","{'positive': 7, 'trust': 3, 'anticipation': 5, 'joy': 2, 'anger': 2, 'negative': 5, 'disgust': 1, 'fear': 2, 'sadness': 3, 'surprise': 2}",True,True,True,i absolutely adored karens new book quotto tame a highland warriorquot it was a book that i truly could put_NEG down karen writes in such a way that quotbamquot she grabs your attention in the very beginning and pulls you right into the story itself while reading this novel i felt a real familiarity with the two main characters grimm and jillian she really gets into setting up the background but doesnt pound it background info that is to death like so many other authors do so much that you want to put the book down right then for her characters so that you feel like you know them personally you can really quotfeelquot the characters emotions and the conflicts going on between them i have to say that its so very easy with karen monings writing style to get totally caught up and lost in the story line when i read her first book quotbeyond the highland mistquot i didnt think that a romance novel could get any better boy was i wrong this novel in my opinion far surpasses the last which like i said above didnt think could get any better i really loved everything about her latest novel especially the way she grabs your attention from the start and keeps it the whole way through cant wait to read her next one
1,0156078805,Architecture Of The Arkan

# Add negated token to negated emotion

In [59]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data files
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
    
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk.corpus import wordnet

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Function to preprocess text with negation handling and lemmatization
def preprocess_text_with_negation_and_lemmatization(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    
    # Handle negations
    negation_words = set(['not', 'no', 'never', 'none'])
    negated = False
    processed_tokens = []
    
    for token in tokens:
        if token in negation_words:
            negated = True
            continue
        
        lemma = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        
        if negated:
            processed_tokens.append(lemma + '_NEG')
            negated = False
        else:
            processed_tokens.append(lemma)
    
    return ' '.join(processed_tokens)

# Load datasets
books_df = pd.read_csv('books.csv')
reviews_df = pd.read_csv('reviews.csv')

reviews_df['cleaned_review'] = reviews_df['review'].apply(preprocess_text_with_negation_and_lemmatization)
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk.corpus import wordnet

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Function to preprocess text with negation handling and lemmatization
def preprocess_text_with_negation_and_lemmatization(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    
    # Handle negations
    negation_words = set(['not', 'no', 'never', 'none'])
    negated = False
    processed_tokens = []
    
    for token in tokens:
        if token in negation_words:
            negated = True
            continue
        
        lemma = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        
        if negated:
            processed_tokens.append(lemma + '_NEG')
            negated = False
        else:
            processed_tokens.append(lemma)
    
    return ' '.join(processed_tokens)

# Load datasets
books_df = pd.read_csv('books.csv')
reviews_df = pd.read_csv('reviews.csv')

reviews_df['cleaned_review'] = reviews_df['review'].apply(preprocess_text_with_negation_and_lemmatization)

# Function to preprocess text with negation handling and lemmatization
def preprocess_text_with_negation_and_lemmatization(text):
    text= remove_stop_words(text)
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    tokens = [word for word in tokens ]
    
    # Handle negations
    negation_words = set(['not', 'no', 'never', 'none'])
    negated = False
    processed_tokens = []
    
    for token in tokens:
        if token in negation_words:
            negated = True
            continue
        
        lemma = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        
        if negated:
            processed_tokens.append(lemma + '_NEG')
            negated = False
        else:
            processed_tokens.append(lemma)
    
    return ' '.join(processed_tokens)
# Load the expanded emotion lexicon
expanded_lexicon = {}
with open('expanded_emotions.txt') as f:
    for line in f:
        word, emotion = line.strip().split(': ')
        expanded_lexicon[word.strip("'")] = emotion.strip("',")

# Function to classify text based on the expanded lexicon
def classify_emotions_with_negation(text):
    emotions = []
    for word in text.split():
        if word in expanded_lexicon:
            emotions.append(expanded_lexicon[word])
        elif word.endswith('_NEG') and word[:-4] in expanded_lexicon:
            # Handle negated words
            original_emotion = expanded_lexicon[word[:-4]]
            negated_emotion = f"not_{original_emotion}"
            emotions.append(negated_emotion)
    return emotions

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [64]:
sample_sentences = [
    "I am happily",
    "I am not happy",
    "I enjoying this book",
    "is none happy"
]

for sentence in sample_sentences:
    processed = preprocess_text_with_negation(sentence)
    emotions = classify_emotions_with_negation(processed)
    print(f"Sentence: {sentence}")
    print(f"Processed: {processed}")
    print(f"Emotions: {emotions}\n")

Sentence: I am happily
Processed: i am happily
Emotions: []

Sentence: I am not happy
Processed: i am happy_NEG
Emotions: ["not_happy',"]

Sentence: I enjoying this book
Processed: i enjoying this book
Emotions: []

Sentence: is none happy
Processed: is happy_NEG
Emotions: ["not_happy',"]



In [65]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Verify lemmatization
words = ["enjoyed", "enjoying", "enjoys", "enjoyable"]
lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
print("Lemmatized Words:", lemmatized_words)




Lemmatized Words: ['enjoy', 'enjoy', 'enjoys', 'enjoyable']


In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk.corpus import wordnet
from collections import Counter

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Function to preprocess text with negation handling and lemmatization
def preprocess_text_with_negation_and_lemmatization(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    
    # Define the negative stop words that you want to exclude
    negative_stop_words = set(['not'])
    
    # Remove the negative stop words from the stop words set
    final_stop_words = stop_words - negative_stop_words
    tokens = [word for word in tokens if word not in final_stop_words]
    
    # Handle negations
    negation_words = set(['not', 'no', 'never', 'none'])
    negated = False
    processed_tokens = []
    
    for token in tokens:
        if token in negation_words:
            negated = True
            continue
        
        lemma = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        
        if negated:
            processed_tokens.append(lemma + '_NEG')
            negated = False
        else:
            processed_tokens.append(lemma)
    
    return ' '.join(processed_tokens)

#Load datasets
data['cleaned_review_v2'] = data['review/text'].apply(preprocess_text_with_negation_and_lemmatization)

# Load the expanded emotion lexicon
expanded_lexicon = {}
with open('expanded_emotions.txt') as f:
    for line in f:
        word, emotion = line.strip().split(': ')
        expanded_lexicon[word.strip("'")] = emotion.strip("',")

# Function to classify text based on the expanded lexicon
def classify_emotions_with_negation(text):
    emotions = []
    for word in text.split():
        if word in expanded_lexicon:
            emotions.append(expanded_lexicon[word])
        elif word.endswith('_NEG') and word[:-4] in expanded_lexicon:
            # Handle negated words
            original_emotion = expanded_lexicon[word[:-4]]
            negated_emotion = f"not_{original_emotion}"
            emotions.append(negated_emotion)
    return emotions


data['expandedemotions.txt'] = data['cleaned_review_v2'].apply(classify_emotions_with_negation)

data['expanded_emotion_counts'] = data['expandedemotions.txt'].apply(Counter)

sample_sentences = [
    "I enjoyed this book",
    "I am not happy",
    "i am strain",
    "i am not strain",
    "I never enjoyed this book",
    
]

for sentence in sample_sentences:
    processed = preprocess_text_with_negation_and_lemmatization(sentence)
    emotions = classify_emotions_with_negation(processed)
    print(f"Sentence: {sentence}")
    print(f"Processed: {processed}")
    print(f"Emotions: {emotions}\n")


Sentence: I enjoyed this book
Processed: enjoy book
Emotions: ['attached']

Sentence: I am not happy
Processed: happy_NEG
Emotions: ['not_happy']

Sentence: i am strain
Processed: strain
Emotions: ['anxious']

Sentence: i am not strain
Processed: strain_NEG
Emotions: ['not_anxious']

Sentence: I never enjoyed this book
Processed: enjoy_NEG book
Emotions: ['not_attached']



In [101]:
data

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,...,sentiment,sentiment_scores,emotions,emotion_counts,emotions_nrc,sentiment _check,emotion_check,cleaned_review_v2,expandedemotions.txt,expanded_emotion_counts
0,156787326,A Room of One's Own,NaN,A3U5AH52JL10DV,Eric Chuang,54/65,5,945129600,Provision for Feminist Future,"70 years have passed since the first publication of &quot;A Room of One's Own,&quot; and yet we have not seen as many dramatic changes, let alone improvements, in social mentality as early feminist thinkers like Virginia Woolf wished to provoke to establish women's new role as equal to men's. She argued that the reason why there was no female Shakespeare is not that women are biologically inferior to men but that there was simply no &quot;room&quot; for women to develop themselves, both metaphorically and realistically speaking. Therefore, in this book she encourages women to have a room of their own and a stable income to ensure a career. However, women, in the past as well as the present, have long been &quot;grounded&quot; by men. For a woman to have a room of her own and a stable income means that the woman is invading (from male chauvinistic viewpoint) men's territory, and this kind of behavior (and thoughts) is not to be allowed in male-dominated societies.Early feminist thinkers like Virginia Woolf provided later generations with iron-cast proof (as far as I'm concerned) that women are no &quot;second sex&quot; by pointing out the false discriminations men put against women for men's own convenience. (Ironically, I see men suffer as well from doing so.) Thinkers like Virginia Woolf provided &quot;rooms&quot; to develop feminist thoughts, and these rooms also provoke controversies and debates because feminist way of thinking is revolutionary. At any rate, there would be no improvements of women's role in society if there were no Virginia Woolf and other first-wave feminist thinkers.At the end of the twentieth century, in spite of the burgeoning &quot;industry&quot; of feminism, the real condition of women appears to be quite depressing. The real condition of women goes like this: &quot;During the last decades women's representation in education has grown enormously but so has our [their] participation in low paid and part-time work. So that, for example, the percentage of women in German higher education has doubled yet the degree of confidence German women express for women in non-traditional jobs is one of the lowest in Europe...,&quot; and &quot;similarly feminist literary criticism has created a lively and substantial body of work in the last decades but continues to exist in a hostile and often marginal academic place.&quot; (&quot;A Reader's Guide to Contemporary Feminist Literary Criticism&quot;, 1994: 290) One cannot fail to see (if one is willing to open his or her eyes) that we have made just a tiny bit of progress since the first-wave feminism. There is room for improvement, indeed, but people's ignorance of women's real position in society, women's subordinate educational, economical, and political conditions, and the overall social status of women being secondary, must first be recognized. In this sense, Virginia Woolf's &quot;A Room of One's Own&quot; appears to be especially inspiring.In my opinion, feminism is not only for or about women, it is also for and about men, because the world is composed of both sexes, and men suffer (without understanding, because of stupidity) from the traditional, male chauvinistic attitude as well. What is important is that feminist way of thinking is a breakthrough, revolutionary philosophy that challenges the way we perceive the world for centuries. &quot;A Room of One's Own&quot; opens my mind's eye; it is, no doubt, a classic that must be read.",...,roberta_neutral,0.630940,"[' adequate', ' attracted', ' happy', ' happy', ' sad', ' angry', ' alone', ' angry', ' attracted', ' sad']","Counter({' att